# __Dependencies__

In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow.python as tf
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.constraints import max_norm
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

# __Marking Keypoints With MP Holistic__

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
#dot, line

In [5]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        #flips frame horizontally --fixes inversed landmarks
        #frame = cv2.flip(frame, 1)

        # Check if frame reading was successful
        if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            print("Error: Failed to capture a valid frame from the webcam.")
            break

        # Print the shape of the frame for further diagnostics
        #print("Frame shape:", frame.shape)
        
        #Make detections (assuming you have the function implemented)
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# __Extracting Keypoints__

In [6]:
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# __Creating File Directory__

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['ILoveYou', 'Okay', 'Peace'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [9]:
actions

array(['ILoveYou', 'Okay', 'Peace'], dtype='<U8')

In [10]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
#HandRecognition\MP_Data

# __Data Collection__

In [33]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [20]:
cap.release()
cv2.destroyAllWindows()

# __Pre-Procees Data__

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'ILoveYou': 0, 'Okay': 1, 'Peace': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
x = np.array(sequences)

In [15]:
y = to_categorical(labels).astype(int)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

# __Build and Train LSTM Neural Network__

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(BatchNormalization())
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(actions.shape[0], activation='softmax'))


In [65]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
def schedule(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(schedule)
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback, lr_scheduler])

Epoch 1/2000
3/3 [==============================] - 4s 34ms/step - loss: 1.6697 - categorical_accuracy: 0.3059 - lr: 0.0010
Epoch 2/2000
3/3 [==============================] - 0s 37ms/step - loss: 1.5622 - categorical_accuracy: 0.3176 - lr: 0.0010
Epoch 3/2000
3/3 [==============================] - 0s 34ms/step - loss: 1.1021 - categorical_accuracy: 0.4824 - lr: 0.0010
Epoch 4/2000
3/3 [==============================] - 0s 32ms/step - loss: 0.9722 - categorical_accuracy: 0.5176 - lr: 0.0010
Epoch 5/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.9515 - categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 6/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.7988 - categorical_accuracy: 0.6118 - lr: 0.0010
Epoch 7/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.7424 - categorical_accuracy: 0.6471 - lr: 0.0010
Epoch 8/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.9573 - categorical_accuracy: 0.6118 - lr: 0.0010
Epoch 9/

In [62]:
model.save('action.keras')

In [19]:
model.load_weights('action.keras')

In [61]:
del model

# __Make Predictions__

In [20]:
res = model.predict(x_test)

1/1 [==============================] - 0s 459ms/step


In [21]:
actions[np.argmax(res[2])]

'Peace'

In [22]:
actions[np.argmax(y_test[4])]

'ILoveYou'

In [23]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 26ms/step


In [24]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [25]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [26]:
accuracy_score(ytrue, yhat)

1.0

# __Test in Real Time__

In [27]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    for num, prob in enumerate(res):
        if num < len(colors):  # Check if num is a valid index for the colors list
            cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame


In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.6

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        #flips frame horizontally --fixes inversed landmarks
        #frame = cv2.flip(frame, 1)
        #with flip : no peace, without flip : no ILY
        
        # Check if frame reading was successful
        if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            print("Error: Failed to capture a valid frame from the webcam.")
            break

        # Print the shape of the frame for further diagnostics
        #print("Frame shape:", frame.shape)
        
        #Make detections (assuming you have the function implemented)
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        # Draw landmarks
        draw_styled_landmarks(image, results)

         # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        #print("Raw predictions:", res)
        #print("Predicted action index:", np.argmax(res))
        #print("Predicted action:", actions[np.argmax(res)])


        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 23ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 22ms/step
Peace
1/1 [==============================] - 0s 21ms/step
Peace
1/1 [==============================] - 0s 24ms/step
Peace
1/1 [==============================] - 0s 23ms/step
Peace
1/1 [==============================] - 0s 23ms/step
Peace
1/1 [=========

In [31]:
label_map

{'ILoveYou': 0, 'Okay': 1, 'Peace': 2}

In [29]:
actions

array(['ILoveYou', 'Okay', 'Peace'], dtype='<U8')